# Creating a data set for model training from 2021 season data

In [15]:
#Imports
import statsapi
import datetime 
import pandas as pd
import json

In [2]:
#Getting start date and end date of a season
def season_start_end(year):
    season_info = statsapi.get("seasons", {"sportId": 1, "season": year})
    season_start = season_info['seasons'][0]['regularSeasonStartDate']
    season_end = season_info['seasons'][0]['regularSeasonEndDate']
    return [season_start, season_end]

In [3]:
start_end_2021 = season_start_end(2021)
season_start = start_end_2021[0]
season_end = start_end_2021[1]

In [4]:
#Adding 35 days to the season start, when I want to start collecting data
season_start_offset = (datetime.datetime.strptime(season_start, "%Y-%m-%d") + datetime.timedelta(days=35)).strftime('%Y-%m-%d')

In [5]:
#Geting all season matchup outcomes
def season_match_outcomes(season_start, season_end):
    dict_list = []
    for i in statsapi.schedule(start_date=season_start, end_date=season_end):
        match_dict = {}
        try:
            match_dict['game_id'] = i['game_id']
            match_dict['win'] = i['winning_team']
            match_dict['loss'] = i['losing_team']
            dict_list.append(match_dict)
        except: pass
    return dict_list

In [6]:
#Setting variable to the output for season match outcomes
all_matches = season_match_outcomes(season_start_offset, season_end)

In [7]:
#Creating a list of all game ids for period in 2021 season
game_ids = [i['game_id'] for i in all_matches]

In [14]:
game_info = statsapi.boxscore_data(632520, timecode=None)
game_info

{'gameId': '2021/09/13/sdnmlb-sfnmlb-1',
 'teamInfo': {'away': {'id': 135,
   'abbreviation': 'SD',
   'teamName': 'Padres',
   'shortName': 'San Diego'},
  'home': {'id': 137,
   'abbreviation': 'SF',
   'teamName': 'Giants',
   'shortName': 'San Francisco'}},
 'playerInfo': {'ID624428': {'id': 624428,
   'fullName': 'Adam Frazier',
   'boxscoreName': 'Frazier, A'},
  'ID543339': {'id': 543339,
   'fullName': 'Daniel Hudson',
   'boxscoreName': 'Hudson, D'},
  'ID595777': {'id': 595777,
   'fullName': 'Jurickson Profar',
   'boxscoreName': 'Profar'},
  'ID642731': {'id': 642731,
   'fullName': 'Thairo Estrada',
   'boxscoreName': 'Estrada'},
  'ID605397': {'id': 605397,
   'fullName': 'Joe Musgrove',
   'boxscoreName': 'Musgrove'},
  'ID657277': {'id': 657277, 'fullName': 'Logan Webb', 'boxscoreName': 'Webb'},
  'ID489334': {'id': 489334,
   'fullName': 'Craig Stammen',
   'boxscoreName': 'Stammen'},
  'ID500779': {'id': 500779,
   'fullName': 'Jose Quintana',
   'boxscoreName': 'Quin

In [9]:
#Takes a long time to run (16min) - There was a better way I am sure, a df would have been better, but using this for now
#Commenting out to not run again
#dict_list = []
#for i in game_ids:
#    game_info = statsapi.boxscore_data(i, timecode=None)
#    roster_dict = {}
#    roster_dict['game_id'] = int(i)
#    roster_dict['home_roster'] = game_info['home']['battingOrder'] + game_info['home']['bullpen']
#    roster_dict['away_roster'] = game_info['away']['battingOrder'] + game_info['away']['bullpen']
#    dict_list.append(roster_dict)
#dict_list

In [10]:
#Creating json from above dict_list
#Commenting out to not run again
#with open('roster_dict', 'w') as fout:
#    json.dump(dict_list, fout)

## Creating Player Stats function

###### Work in progress, not quite done yet

In [46]:
#Create function to get player stats
#Inputs are the id, start date and end date

#Currently not getting all the fielding stats, as there are more than one set of them - based on the pos they play - fix in progress

#Updates:
# Need to make 3 API calls, one for hitting, fielding, and pitching.  Might need to add an if statement for pitching/fielding based on position, we will see
# use the add_dicts function to sum all the fielding stats for each position played, as they are split
# Store them all in an organized dictionary for later use

def player_stats(id,start_date, end_date, season):
    player_info = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate={start_date},endDate={end_date},season={season})"})
    #dict where I will add all the collected stats
    stat_dict = {}
    #Temporary storage for fielding dicts for all positions
    fielding_list = []
    #List to check if the stats for a fielding position have already been collected - as they are listed more than once
    pos_codes = []
    try:
        player_hitting_stats = player_info['people'][0]['stats'][0]['splits'][0]['stat']
        stat_dict['hitting'] = player_hitting_stats
    except: pass
    try: 
        for i in player_info['people'][0]['stats'][1]['splits']:
            if player_info['people'][0]['stats'][1]['splits'][i]['stat']['position']['code'].isin(pos_codes):
                pass
            else:
                pos_codes.append(player_info['people'][0]['stats'][1]['splits'][i]['stat']['position']['code'])
                fielding_list.append(player_info['people'][0]['stats'][i]['splits'][0]['stat']) 
        all_fielding = {}
        for i in fielding_list:
            all_fielding.update(i)

            stat_dict['fielding'] = 1 #Placeholder
    except: pass
    try:
        player_pitching_stats = player_info['people'][0]['stats'][2]['splits'][0]['stat']
        stat_dict['pitching'] = player_pitching_stats
    except:pass
    return stat_dict

#### Various tests and examples trying to figure out how to get the above function working

In [ ]:
for i in statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][1]['splits']:
    print(i)

In [ ]:
statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][1]['splits'][0]['stat']['position']['code']


In [ ]:
statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][1]['splits'][0]['stat']

In [152]:
#Testing to see if I can combine fielding stats into a dict
#It works except for the 'position' dictionary, which I do not need the info from
#Also combines fielding as a string, but I can fix that later, if I decide to use that stat (thinking I will not for now)
d1 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][0]['stat']

d2 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][1]['stat']

d3 = dict(d1) # don't do `d3=d1`, you need to make a copy

d3.update(d2) 
try:
    for i, j in d1.items():

        for x, y in d2.items():

            if i == x:

                d3[i]=(j+y)
except: pass
print(d3)

{'gamesPlayed': 61, 'gamesStarted': 51, 'assists': 89, 'putOuts': 212, 'errors': 1, 'chances': 302, 'fielding': '1.000.995', 'position': {'code': '3', 'name': 'First Base', 'type': 'Infielder', 'abbreviation': '1B'}, 'rangeFactorPerGame': '6.81', 'rangeFactorPer9Inn': '8.20', 'innings': '202.0', 'games': 27, 'doublePlays': 10, 'triplePlays': 0, 'throwingErrors': 0}


In [150]:
#Testing for three dictionaries
d1 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[fielding],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][0]['stat']

d2 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][1]['stat']

d0 = statsapi.get("people", {"personIds": 518934, "hydrate": "stats(group=[hitting,fielding,pitching],type=[byDateRange],\
    startDate=2022-04-05,endDate=2022-08-12,season=2022)"})['people'][0]['stats'][0]['splits'][2]['stat']

d3 = dict(d1) # don't do `d3=d1`, you need to make a copy

d3.update(d2) 
try:
    for i, j in d1.items():

        for x, y in d2.items():

                if (i == x):

                    d3[i]=(j+y)
except: pass
d4 = dict(d3)
d4.update(d0)
try:
    for i, j in d3.items():

        for x, y in d0.items():

                if (i == x):

                    d4[i]=(j+y)
except: pass
print(d4)

{'gamesPlayed': 65, 'gamesStarted': 55, 'assists': 89, 'putOuts': 212, 'errors': 1, 'chances': 302, 'fielding': '1.000.995.000', 'position': {'code': '10', 'name': 'Designated Hitter', 'type': 'Hitter', 'abbreviation': 'DH'}, 'rangeFactorPerGame': '0.00', 'rangeFactorPer9Inn': '-.--', 'innings': '0.0', 'games': 4, 'doublePlays': 0, 'triplePlays': 0, 'throwingErrors': 0}


In [143]:
#Making a function to shorten the code - will use this in the stats function to get all the relevant fielding stats added together
def add_dicts(d1, d2):
    d3 = dict(d1)
    d3.update(d2)
    try:
        for i, j in d1.items():

            for x, y in d2.items():

                if (i == x):

                    d3[i]=(j+y)
    except: pass
    return d3

In [ ]:
#Testing function for 2020
player_stats(518934,'2022-04-05','2022-08-12', 2022)

In [ ]:
#reading json
f = open('roster_dict')
data = json.load(f)